In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import matplotlib.pyplot as plt

from dataloader.rosen import RosenData
from visualize import get_vae, get_model, set_random, build_estimator
config = {
    'estimator': 'nngp',
    'random_seed': 43,
    'n_dim': 10,
    'data_size': 2000,
    'data_split': [0.2, 0.8, 0, 0],
    'update_size': 100,
    'al_iterations': 10,
    'verbose': True,
    'use_cache': True,
    'layers': [10, 128, 64, 32, 1],
    'patience': 5,
    'retrain': True,
    'model_path': 'model/data/rosen_visual.ckpt',
    'epochs': 20_000,
}


In [ ]:
rosen = RosenData(
    config['n_dim'], config['data_size'], config['data_split'],
    use_cache=config['use_cache'])

x_pool, y_pool = rosen.dataset('pool')
x_train, y_train = rosen.dataset('train')
x_val, y_val = rosen.dataset('train')

set_random(config['random_seed'])
model = get_model(
    config['layers'], config['retrain'], config['model_path'],
    (x_train, y_train), (x_val, y_val), epochs=30_000)

In [ ]:
num = 10
prediction = model(x_val).cpu().numpy()
for x, y in zip(prediction[-num:], y_val[-num:]):
    print(x, y)


In [ ]:
estimator = build_estimator('nngp', model)
estimations = estimator.estimate(x_val, x_train, y_train)
predictions = model(x_val).cpu().numpy()
error = np.abs(predictions-y_val)/(predictions+y_val)
plt.figure(figsize=(12, 9))
plt.ylabel('Uncertainty')
plt.xlabel('Error')
plt.scatter(error, estimations)


In [ ]:
from analysis.metrics import uq_accuracy

error = np.abs(predictions - y_val)
uq_accuracy(estimations, error, 0.05)


In [3]:
import numpy as np
import matplotlib.pyplot as plt 

import pandas as pd  
import seaborn as sns 

from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split

In [4]:
dataset = load_boston()
print(dataset.keys())
dataset['target'].shape

In [5]:
layers = [13, 64, 32, 16, 1]
x_train, x_val, y_train, y_val = train_test_split(dataset['data'], dataset['target'], test_size=0.2)
print(x_train.shape, y_train.shape)

In [ ]:
retrain = True
from model.mlp import MLP
import torch

model_path = 'model/data/boston.ckpt'
model = MLP(layers, l2_reg=1e-3)
if retrain:
    model.fit((x_train, y_train), (x_val, y_val), epochs=30000, patience=10)
    torch.save(model.state_dict(), model_path)
else:
    model.load_state_dict(torch.load(model_path))
    model.eval()


In [7]:
predictions = model(x_val).cpu().numpy()
for x, y in zip(y_val[-10:], predictions[-10:]):
    print(x, y)

# estimator = build_estimator('nngp', model)
# estimations = estimator.estimate(x_vak, x_train, y_train)

# error = np.abs(predictions-y_val)/(predictions+y_val)
# plt.figure(figsize=(12, 9))
# plt.ylabel('Uncertainty')
# plt.xlabel('Error')
# plt.scatter(error, estimations)
